In [2]:
# Charges protocol fees for a given pool and token -> this function should be executed before any operation that touches this token
# static_balance = balance of a given pool for a given token
# rate = rate of token
# last_live_balance = live balance (i.e. static_balance * rate) of this token and pool AFTER the fee was last charged 
# protocol_fee_percentage = % of fees charged by protocol
## PS: static_balance and last_live_balance should be packed in a single 256 word like we do in v2 with managed and cash balances

def calculateProtocolFees(static_balance, rate, last_live_balance, protocol_fee_percentage):
    # Calculate how much in static tokens should be paid as protocol fees
    current_live_balance_without_fees = static_balance * rate
    live_balance_to_charge_fees_on = current_live_balance_without_fees - last_live_balance
    fee_in_static_token = (live_balance_to_charge_fees_on * protocol_fee_percentage) / rate
    
    return fee_in_static_token


# Charges protocol fees for a given pool and token -> this function should be executed before any operation that touches this token
# static_balance = balance of a given pool for a given token
# rate = rate of token
# static_protocol_fee = balance of fee accrued for a given token (doesn't depend on the pool as it's a global value for that specific token)
# last_live_balance = live balance (i.e. static_balance * rate) of this token and pool AFTER the fee was last charged 
# protocol_fee_percentage = % of fees charged by protocol

def chargeProtocolFees(static_balance, rate, static_protocol_fee, last_live_balance, protocol_fee_percentage):
    fee_in_static_token = calculateProtocolFees(static_balance, rate, last_live_balance, protocol_fee_percentage)
    
    # Update states
    static_balance -=fee_in_static_token
    static_protocol_fee += fee_in_static_token
    last_live_balance = static_balance * rate
    
    return (static_balance, static_protocol_fee, last_live_balance)

# View function that should be called to read current live balance of a given pool and token
def readLiveBalance(static_balance, rate, last_live_balance, protocol_fee_percentage):
    fee_in_static_token = calculateProtocolFees(static_balance, rate, last_live_balance, protocol_fee_percentage)
    
    # Update states
    static_balance -=fee_in_static_token
    live_balance = static_balance * rate
    return live_balance



# Basic testing for increasing rates, "Live balance read" should always be equal to new_last_live_balance
def printStats():
    print("Rate:", rate)
    print("Live balance read:", readLiveBalance(static_balance, rate, last_live_balance, protocol_fee_percentage))
    [new_static_balance, new_static_protocol_fee, new_last_live_balance] = chargeProtocolFees(static_balance, rate, static_protocol_fee, last_live_balance, protocol_fee_percentage)
    print("Protocol fees charged")
    print("new_static_balance: ", new_static_balance)
    print("new_static_protocol_fee: ", new_static_protocol_fee)
    print("new_last_live_balance: ", new_last_live_balance)
    print("")

rate = 1
static_balance = 100
static_protocol_fee = 0
last_live_balance = 100
protocol_fee_percentage = 0.5
printStats()

rate = 1.1
printStats()

rate = 1.2
printStats()

Rate: 1
Live balance read: 100.0
Protocol fees charged
new_static_balance:  100.0
new_static_protocol_fee:  0.0
new_last_live_balance:  100.0

Rate: 1.1
Live balance read: 105.0
Protocol fees charged
new_static_balance:  95.45454545454545
new_static_protocol_fee:  4.545454545454551
new_last_live_balance:  105.0

Rate: 1.2
Live balance read: 110.0
Protocol fees charged
new_static_balance:  91.66666666666667
new_static_protocol_fee:  8.333333333333334
new_last_live_balance:  110.0

